<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%AA%A8%EB%8D%B8%EB%A7%812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
gpt = pd.read_csv(f"{DATA_PATH}gpt_4400.csv")
gpt['store'] = ''
gpt['branch'] = ''
gpt.head()

,gpt_review,gpt_reply,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치찌개,,
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,


In [ ]:
gpt.columns = ['고객리뷰','사장댓글','sentiment','food','store','branch']

In [ ]:
tmp = pd.read_csv(f"{DATA_PATH}train+sentiment_0913.csv")
tmp.head()

,고객리뷰,사장댓글,store,branch,food,sentiment
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,,,,1
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,,,,0
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,,,,1
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,,,,0
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,,,,1


In [ ]:
tmp = tmp.loc[tmp.sentiment == 1]
tmp = tmp.sample(frac=1, random_state=SEED)

In [ ]:
tmp = tmp[:4400]

In [ ]:
df = pd.concat([gpt,tmp], axis=0)
df

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치찌개,,
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,
...,...,...,...,...,...,...
32077,맛있게 잘먹었습니다 양도많이 주시고 잘먹었습니다 수고하셨습니다,감사합니다 언제나 행복하세요,1,,,
2122,마라탕만 먹어봤는데 마라샹궈도 너무 맛있어요,맛있게 드셔주셔서 감사합니다~ 요청사항은 조리전 늘 꼼꼼하게 체크하고 있으니 필요하...,1,,,
17408,"맛도 좋고, 음식도 깔끔하고, 양도 많고, 감자도 푹 익어서 맛났어요~",고객님 맛있게 만족하며 드셔주셔서 정말 진심으로 감사드립니다 늘 행복한 한끼식사로 ...,1,,,
14705,만족스러운 김치찜 집을 찾아서 너무 좋아요~,토핑폭탄을 이용해 주셔서 너무 감사드립니다. 바쁜시간에도 저희가게에 리뷰선물까지 주...,1,김치찌개,토핑폭탄,


# csv 불러오기

## 테스트 데이터셋 분리

In [ ]:
yogiyo = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv")
yogiyo

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,저도 모르게 은근 여기 자주 시켜먹고 있었네요ㅎ \n늘 잘먹고 있습니다,5,5,5,5.0,저희 메뉴 맛있죠?ㅎㅎ 이렇게 빠져 드시는 거에요>.<\n\n그냥 일로서 만드는 게...,제육옛날도시락＋김찌 정식＋（서비스）콜라
1,1988응답하라추억의옛날도시락-방이점,김치찌개 된장찌개 잡채\n다 잘먹었습니다,5,5,5,5.0,전부 잘 드셔 주셔서 감사합니다 ><\n\n좋은 시간을 드리고 싶었어요ㅠㅠ 열심히 ...,"돼지고기김치찌개＋대접밥,우삼겹된장찌개＋대접밥,오색빛깔 잡채"
2,1988응답하라추억의옛날도시락-방이점,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요. 다음에도 또시킬듯요,5,5,5,5.0,안녕하세요~~! :D\n\n저희 매장을 처음 찾아주신 고객님이시네요😆 처음 방문이라...,"[불맛]제육 옛날도시락,우삼겹옛날도시락＋미니냉면＋（서비스）콜라"
3,1988응답하라추억의옛날도시락-방이점,맛있습니다! 근데 국물이 안와서 아쉽습니다..,4,4,4,3.0,안녕하세요 1988응답하라추억의옛날도시락입니다.\n\n저희 측 실수로 인하여 누락이...,양념갈비 옛날도시락
4,1988응답하라추억의옛날도시락-방이점,맛있게잘먹었습니다,5,5,5,5.0,"헤헤, gh**님! 감사해요! 😊 \n\n맛있게 드셨다니 앞으로도 더 열심히 일할게...","삼겹정찬도시락,돼지고기김치찌개＋대접밥"
...,...,...,...,...,...,...,...,...
24993,놀부 부대찌개-광주우산점,먹을만하다고느낍니다,4,3,3,3.0,놀부를 이용해 주셔서 감사합니다 ^^\n큰 만족을 드리지 못한것 같아 죄송합니다.\...,NaN
24994,놀부 부대찌개-광주우산점,조리를해야하는 불편함이 있긴했지만..매장에서 먹는맛 그대로이고 양도 엄청많네요 4인...,5,5,5,5.0,저희 놀부를 이용해 주셔서감사드립니다 ^^\n집에서 조리시에도 매장과 최대한 비슷하...,NaN
24995,놀부 부대찌개-광주우산점,오랜만에 놀부부대찌게~~~\n맛있었는더ㆍ\n햄 양을 늘리심 좋을듯,5,5,4,5.0,놀부를 이용해 주셔서 감사합니다 ^^\n맛있게 드셨다니 다행입니다.\n저희는 정해진...,NaN
24996,놀부 부대찌개-광주우산점,예전에 참전한 6.25 에서 먹은 부대찌개 맛이 나네요 맛있읍니다.,5,5,5,5.0,저희 놀부부대찌개를 맛있게 드셔주셔서 감사합니다 ^^\n어디서 드시더라도 항상 맛있...,NaN


In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(yogiyo, test_size=0.2, random_state=SEED, stratify=yogiyo['별점'])

## 정제된 학습 데이터셋 불러오기

In [ ]:
ng_df = pd.read_csv(f"{DATA_PATH}0908_ng_df.csv")
ng_df

,고객리뷰,사장댓글,sentiment,store,branch,food
0,"야채,단무지 넣지 말라고 요청 드렸는데 주먹밥에 들어거는 김까지 안보내셨네요 담엔 ...",먼저 죄송 합니다 바로 전화 주셨으면 김가루 가져다드릴건데 죄송 합니다 재주문시 꼭...,0,NaN,,야채
1,폴드포크 제가 제일 좋아하는 메뉴입니다~! 근데 울트라 치즈는 어디에.? 보이지도 ...,고객님 안녕하세요~ 우선 울트라치즈에 치즈맛이 안느껴지셨다니 .치즈가 세번들어가는 ...,0,NaN,,무나
2,아파서 시켰어요 저는 참깨김후레이크? 그거가 저랑 잘 안맞네요.,안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,0,NaN,,
3,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십년만에 주문을 해봅니다. 양 넉...,안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,0,NaN,,
4,국은 시원하게 맛있었어요 국물양이 더 많았으면 더 좋았겠습니다,맛있게 드셨다니 감사드립니다. 매장과 같이 최대한 드리고있습니다. 배달용기에 국물을...,0,NaN,,
...,...,...,...,...,...,...
1478,예전에는 반찬도 많이 주셨는데 요즘에는 반찬이 없네요,항상 고객분들의 의견에 귀를 기울이고 더 좋은 음식을 준비하기 위해 노력하는 한편 ...,0,NaN,,
1479,부대찌개 맛은 그냥 그런데 속에 든 햄은 꽤 많은데 김치가 좀 모자랐어요 저희 커플...,저희는 항상 10분 안에 모든 메뉴가 준비되어 있지만 배달대행업체의 특성상 동시간대...,0,NaN,,
1480,근데 시간이 너무 오래 걸려요 근데 역씨 놀부는 너무 인기가 많아요,항상 푸짐하고 좋은 음식을 제공해 드리는 곳이 되기 위해 노력하겠습니다.,0,NaN,,
1481,진짜 짜증 나게 하는 게 배달 기사 아저씨가 진짜 최악이에요.,저희 매장을 항상 방문해 주시는 분들께 감사의 마음을 전하며 더 좋은 요리와 따뜻한...,0,NaN,,


In [ ]:
ps_df = pd.read_csv(f"{DATA_PATH}0908_ps_df.csv")
ps_df

,고객리뷰,사장댓글,sentiment,store,branch,food
0,저희는 칼국수 여기에서만 주문해서 먹어요. 국물이 너무 맛있구요 사장님이 친절해서인...,찐~감사요~,1,NaN,,
1,돈가스회관 떡볶이에 돈가스와 치킨가스를 곁들여 먹으니 최상의 조합이네요! 완전 맛있...,고객님 항상 떡볶이를 맛나게 드시는 울 고객님 감사드립니다. 떡볶이와 돈까스가 은근...,1,"돈까,돈까스회,돈까스회관",,떡볶
2,2년만에 다시 먹을 수 있으니 너무 벅차올 거 같아요 코로나 젤 심에 시기에 가장 ...,저팔계 중국 마라탕 덮밥 요리 입니다. 저희 음식을 맛있게 드시고 소중한 사진리뷰...,1,"저팔,저팔계",,덮밥
3,빽다방은 늘 옳아요~,"안녕하세요. 대전역점 빽다방입니다. 맛있게 드셨다니 기분이 좋아요 싸고, 크고, 맛...",1,"빽다,빽다방",대전역점,
4,매주 이용하는곳 입니다,감사합니다 리뷰님.,1,NaN,,
...,...,...,...,...,...,...
2995,"맛, 서비스 친절 모두 최고에요. 양도 넉넉하고 사이드 메뉴 구성도 좋습니다.",아직 부족한게많은데 웃으면서 이해해주시니 감사할따름입니다! 다음에도 고객님의 만족...,1,홍스,,
2996,맛있게 먹었어요.,감사합니다 항상 변함없는 맛으로 찾아 뵙겠습니다,1,NaN,,
2997,맛있어요,맛있게 드셔주시고 시간 내서 소중한 리뷰 달아주셔서 너무너무 감사합니다 ~.~ 고객...,1,NaN,,
2998,튀김류가 맛있어요,맛있게 드셔 주셔서 감사합니다. 소중한 리뷰도 감사드리고 앞으로도 많은 관심과 이용...,1,국수나무,,국수


In [ ]:
ng_df['store'] = ng_df['store'].fillna("")
ps_df['store'] = ps_df['store'].fillna("")

### 키워드 사용할 경우

In [ ]:
df = pd.read_csv(f"{DATA_PATH}0906.csv")
df

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews,keybert_reviews
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931,"['배달', '감사', '맛있']","[('배달', 0.8359), ('감사', 0.8353), ('맛있', 0.4462)]"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811,"['세트', '많', '좋', '맛있']","[('세트', 0.806), ('맛있', 0.5224)]"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,NaN,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962,['맛있'],"[('맛있', 1.0)]"
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,NaN,,,항상 맛있어요 서비스도 최고,1,0.801292,"['맛있', '서비스', '최고']","[('서비스', 0.9012), ('최고', 0.5062), ('맛있', 0.5062)]"
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,NaN,,,모든 반찬이 다 맛있습니다!,1,0.762959,"['반찬', '맛있']","[('반찬', 0.9452), ('맛있', 0.4905)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20719,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4,맛있게 잘 먹었습니다.,더 노력하는 보둘 미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 ...,NaN,,미역,맛있게 잘 먹었습니다.,1,0.519741,['맛있'],"[('맛있', 1.0)]"
20720,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...,NaN,,라면,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,0,0.542860,"['국물', '많', '국물', '같', '시원하']","[('시원하', 0.7763), ('국물', 0.7322)]"
20721,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,NaN,,,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,1,0.766628,"['맛있', '포장', '무엇', '반찬', '짜', '맛있', '구성', '가지'...","[('구성', 0.5366), ('만족', 0.4777), ('반찬', 0.4172..."
20722,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,NaN,,,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,1,0.748113,"['많', '덕분', '맛있']","[('덕분', 0.9048), ('맛있', 0.4602)]"


In [ ]:
df = df[['keybert_reviews','corrected_reply','store','branch','food','sentiment']]

In [ ]:
df["corrected_reply"].isnull().sum()

33

In [ ]:
df[df.corrected_reply.isna()]

,keybert_reviews,corrected_reply,store,branch,food,sentiment
498,"[('감사', 0.8251), ('서비스', 0.7837), ('맛있', 0.3705)]",NaN,NaN,,,1
1192,"[('용기', 0.5263), ('플라스틱', 0.4466), ('나머지', 0.3...",NaN,NaN,,,0
1387,"[('방법', 0.7218), ('다르', 0.5836), ('포장', 0.5808...",NaN,NaN,,,0
1641,[],NaN,NaN,,,1
1737,"[('설정', 0.485), ('한마디', 0.3867), ('부모', 0.3831...",NaN,NaN,,,0
1740,"[('만큼', 0.9114)]",NaN,NaN,,,1
2357,"[('풍성', 0.5414), ('여기', 0.4246), ('넙대', 0.2949...",NaN,NaN,,,1
4424,"[('음식', 0.8082), ('배달', 0.7277), ('맛있', 0.5014)]",NaN,NaN,,,1
4711,"[('국밥', 0.5312), ('내용물', 0.5099), ('알차', 0.438...",NaN,NaN,,,1
6018,"[('소스', 0.674), ('여기', 0.6608), ('꿔바로우', 0.497...",NaN,NaN,,,1


In [ ]:
df = df.dropna(subset=['corrected_reply'])

In [ ]:
df["keybert_reviews"].isnull().sum()

0

In [ ]:
df.columns = ["고객리뷰","사장댓글",'store','branch','food','sentiment']
df

,고객리뷰,사장댓글,store,branch,food,sentiment
0,"[('배달', 0.8359), ('감사', 0.8353), ('맛있', 0.4462)]","안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,1
1,"[('세트', 0.806), ('맛있', 0.5224)]",놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,1
2,"[('맛있', 1.0)]",은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,NaN,,,1
3,"[('서비스', 0.9012), ('최고', 0.5062), ('맛있', 0.5062)]",주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,NaN,,,1
4,"[('반찬', 0.9452), ('맛있', 0.4905)]",주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,NaN,,,1
...,...,...,...,...,...,...
20719,"[('맛있', 1.0)]",더 노력하는 보둘 미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 ...,NaN,,미역,1
20720,"[('시원하', 0.7763), ('국물', 0.7322)]",배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...,NaN,,라면,0
20721,"[('구성', 0.5366), ('만족', 0.4777), ('반찬', 0.4172...",보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,NaN,,,1
20722,"[('덕분', 0.9048), ('맛있', 0.4602)]",맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,NaN,,,1


In [ ]:
df['store'] = df['store'].fillna("")

### 긍정/부정 분리

In [ ]:
for i in range(0, 2):
    num_rating = df[df['sentiment'] == i].shape[0]
    print(f"긍/부정 {i}점 개수: {num_rating}, {num_rating / df.shape[0] * 100}%")

긍/부정 0점 개수: 1308, 6.321589096708714%
긍/부정 1점 개수: 19383, 93.6784109032913%


In [ ]:
ng_df = df.loc[df.sentiment == 0]
ng_df

,고객리뷰,사장댓글,store,branch,food,sentiment
19,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,,,야채,0
80,"[('레이크', 0.7232), ('그거', 0.6273), ('아프', 0.626...",안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,,,,0
86,"[('대왕오징어', 0.7619), ('만다린', 0.5751), ('수십', 0....",안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,,,,0
95,"[('시원하', 0.7902), ('국물', 0.7239), ('맛있', 0.4535)]",맛있게 드셨다니 감사드립니다. 매장과 같이 최대한 드리고 있습니다. 배달용기에 국물...,,,,0
111,"[('대통령', 0.8653), ('각하', 0.6785)]",주문 감사합니다~추운 날씨 감기 조심하세요~,,,,0
...,...,...,...,...,...,...
20711,"[('카페', 0.739), ('와플', 0.7204), ('여름', 0.5952)...",더 맛있는 요리로 찾아뵐게요 이번 여름엔 더 맛있게 어떤 요리든 만들어 보겠습니다.,,,,0
20712,"[('냉동', 0.7186), ('오늘', 0.6781), ('선호', 0.5995...",항상 친절한 매장 운영을 위해 노력하겠습니다.,,,,0
20716,"[('거부감', 0.7177), ('생각', 0.6954), ('어묵', 0.655...",그래도 예쁘게 잘 올려주신 사진 덕분에 제가 마음이 놓였습니다 그럼 이만 사진을 봐...,,,,0
20717,"[('배부르', 0.6836), ('쌀국수', 0.628), ('기본', 0.579...",오늘 부족했던 부분 많이 보충해 주셨는데 또 궁금하신 부분이 있으면 언제든 질문해 ...,,,,0


In [ ]:
ng_df = ng_df.copy()

In [ ]:
ps_df = df.loc[df.sentiment == 1]
ps_df

In [ ]:
ps_df = ps_df.copy()

In [ ]:
# df = df.sample(frac=1).reset_index(drop=True) # 데이터 랜덤 섞기
# df

In [ ]:
# lst = [('또먹고싶음', 0.9021), ('맛있러료', 0.9021)]

In [ ]:
# t = []
# for i in lst:
#     tmp = []
#     tmp.append(i[0])
#     t.extend(tmp)
# t

['또먹고싶음', '맛있러료']

In [ ]:
# lst = df['고객리뷰'][0]
# lst

"[('크리스피해서', 0.8349), ('배달', 0.5329), ('빨라요', 0.494)]"

In [ ]:
pat = re.compile("[^가-힣 ]")

keyword = []
for lst in ng_df['고객리뷰']:
    tmp = []
    # print(lst)
    # print(type(lst))
    t = pat.sub("",lst)
    t = t.replace("  ",",")
    tmp.append(t)
    keyword.extend(tmp)

keyword

['주먹밥,요청,단무지,야채 ',
 '레이크,그거,아프,참깨 ',
 '대왕오징어,만다린,수십,주문,예전 ',
 '시원하,국물,맛있 ',
 '대통령,각하 ',
 '드리프트,기사 ',
 '충족,당신,김치,기대 ',
 '완료,문자,어떻 ',
 '옥수수,토핑,추가 ',
 '정도,중간,시간,배달 ',
 '삼키기,평소,배달,국물,맛있 ',
 '곱창,음식,벌레,하나,아쉽 ',
 '콩나물,반찬,원래,편식,계란 ',
 '수저,체크,표시,다음,바쁘 ',
 '구성,참기름,주문,메뉴,덮밥 ',
 '연락,전화,배달,고기,하나 ',
 '요청,감자탕,만족,국물,숟가락 ',
 '소스,떡볶이 ',
 '슬프,선택,이번,메뉴,구매 ',
 '밥알,날치,더블,수준 ',
 '맵기랑,수준,단계,아쉽 ',
 '컨디션,전반,문제,사람,괜찮 ',
 '수저 ',
 '저렴하,주문,가격,프렉스,빠르 ',
 '배달,빠르 ',
 '주문자,실수,이번,다음,매운맛 ',
 '냉면,리뷰,기대,그렇,맛있 ',
 '배달원,노른자,전화,하나,때문 ',
 '당면,이벤트,리뷰,맛있,분모 ',
 '번거롭,음식,기사,배달,그렇 ',
 '배달,국수,괜찮 ',
 '구성,정도,단무지,기본,최소 ',
 '배달,맛있 ',
 '먼지,통화,불만,그것,제품 ',
 '신경,날씨,활어,부분,맛있 ',
 '걱정,부탁,김치,다음,맛있 ',
 '재료,음료,복통,호소,딸내미 ',
 '국물 ',
 '배송,이번,국물,치킨,떡볶이 ',
 '고기,국수,종잇장 ',
 '정도,육수,인분,추가 ',
 '토치,포인트,이번,다음,닭고기 ',
 '미지근하,가락국수,국물 ',
 '피자,치즈 ',
 '고추장,전체,비빔밥,제육,곱빼기 ',
 '몸통,신선도,집게,후기,그렇 ',
 '오랜만,그릇 ',
 '탕수육,전화,소주,배달,어떻 ',
 '요청,말씀,환불,맛있,주문 ',
 '음식,사진,아쉽,양파 ',
 '생선,당황,고기 ',
 '수정,이벤트,배달,카레,뒤늦 ',
 '어떻 ',
 '바비큐,추천,간장,맛있 ',
 '입맛 ',
 '곱창,질기,다음,야채 ',
 '평상시,부드럽,마라,기분,그

In [ ]:
ng_df['keyword'] = keyword
ng_df

,고객리뷰,사장댓글,store,branch,food,sentiment,keyword
19,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,,,야채,0,"주먹밥,요청,단무지,야채"
80,"[('레이크', 0.7232), ('그거', 0.6273), ('아프', 0.626...",안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,,,,0,"레이크,그거,아프,참깨"
86,"[('대왕오징어', 0.7619), ('만다린', 0.5751), ('수십', 0....",안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,,,,0,"대왕오징어,만다린,수십,주문,예전"
95,"[('시원하', 0.7902), ('국물', 0.7239), ('맛있', 0.4535)]",맛있게 드셨다니 감사드립니다. 매장과 같이 최대한 드리고 있습니다. 배달용기에 국물...,,,,0,"시원하,국물,맛있"
111,"[('대통령', 0.8653), ('각하', 0.6785)]",주문 감사합니다~추운 날씨 감기 조심하세요~,,,,0,"대통령,각하"
...,...,...,...,...,...,...,...
20711,"[('카페', 0.739), ('와플', 0.7204), ('여름', 0.5952)...",더 맛있는 요리로 찾아뵐게요 이번 여름엔 더 맛있게 어떤 요리든 만들어 보겠습니다.,,,,0,"카페,와플,여름,그렇"
20712,"[('냉동', 0.7186), ('오늘', 0.6781), ('선호', 0.5995...",항상 친절한 매장 운영을 위해 노력하겠습니다.,,,,0,"냉동,오늘,선호,붕어빵,아쉽"
20716,"[('거부감', 0.7177), ('생각', 0.6954), ('어묵', 0.655...",그래도 예쁘게 잘 올려주신 사진 덕분에 제가 마음이 놓였습니다 그럼 이만 사진을 봐...,,,,0,"거부감,생각,어묵,우리,나라"
20717,"[('배부르', 0.6836), ('쌀국수', 0.628), ('기본', 0.579...",오늘 부족했던 부분 많이 보충해 주셨는데 또 궁금하신 부분이 있으면 언제든 질문해 ...,,,,0,"배부르,쌀국수,기본,추가,계란"


In [ ]:
ps_df['keyword'] = keyword
ps_df

# 모델 불러오기

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.4 MB/s eta 0:00:00


In [3]:
model_name = "skt/kogpt2-base-v2"

In [4]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [ ]:
df.head()

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치찌개,,
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        store = item["store"]
        branch = item["branch"]
        food = item["food"]
        review = item["고객리뷰"]
        # review = item["keyword"]
        answer = item["사장댓글"]

        return {
            "sentiment": sentiment,
            "store": store,
            "branch": branch,
            "food": food,
            "review": review,
            "answer": answer
        }

In [ ]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [
        f"<w>{item['store']}</s><b>{item['branch']}</s><f>{item['food']}</s><q>{item['review']}</s><a>{item['answer']}</s>"
        for item in batch
    ]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [ ]:
dt = ChatDataset(df)
dt[0]

{'sentiment': 0,
 'store': '',
 'branch': '',
 'food': '김치찌개',
 'review': '김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고. ',
 'answer': '안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 위해 노력하겠습니다. 감사합니다.'}

In [ ]:
dt = ChatDataset(df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,     1,  9724,   440,   405,     1,  9724,   444,
            405,  6963,  8420,  8326,  6841,     1,  9724,   455,   405,  6963,
           8420,  8326, 19010, 15126,  8041,  8006, 25856, 10968, 33974,  6920,
          31131, 13875, 40974,  8718,  8006, 25856,  9065,  9313, 15126, 19570,
            389,   739,     1,  9724,   439,   405,  7967,  7114,  8702,  7801,
          25856, 33974,  8326, 24655,  9065,  9399,  9167,  9137, 12615,  9535,
           7415, 11482,  7832, 37194, 18323, 14912, 32272,  9207, 10805, 26616,
          16691, 15940, 37194,     1],
         [ 9724,   461,   405,     1,  9724,   440,   405,     1,  9724,   444,
            405,   739,     1,  9724,   455,   405, 13692, 12371, 27080,  9208,
           8539,  8267,  7724,  8718,  8006, 25856, 10997,  7235, 15126,  8041,
           8006, 25856,   739,     1,  9724,   439,   405,  7609,  8539,  8267,
           7724,  8702,  7816, 16179, 29205, 24721,  6975, 166

In [ ]:
model(**batch["x"]).logits.shape

torch.Size([2, 74, 51200])

# 동결

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"매개변수 {name}는 requires_grad가 True입니다.")
    else:
        print(f"매개변수 {name}는 requires_grad가 False입니다.")

매개변수 transformer.wte.weight는 requires_grad가 True입니다.
매개변수 transformer.wpe.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_1.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_1.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_attn.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_attn.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_proj.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_proj.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_2.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_2.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_fc.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_fc.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_proj.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_proj.bias는 requires_grad가 True입니다.
매개변수 transformer.h.1.ln_1.weight는 requires_grad가 True입니다.
매개변수 transformer.h.1.ln_1.bias는 requires_grad가 True입니다.
매개변수 transformer.h.1.attn.c_attn.weight는 require

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

- 전체 동결

In [ ]:
# class FreezeNet(torch.nn.Module):
#     def __init__(self, model):
#         super().__init__()
#         self.model = model
#         for param in self.model.parameters():
#             param.requires_grad = False
#         self.model.lm_head = torch.nn.Linear(768,51200)

#     def forward(self,x):
#         return self.model(x)

In [ ]:
# model = FreezeNet(model)
# for param in model.parameters():
#     print(param.requires_grad)

- 일부 동결

In [ ]:
for name,param in model.named_parameters():
    if name.startswith("transformer.h.11"):
        break
    param.requires_grad = False

for name, param in model.named_parameters():
    print(name,param.requires_grad)

transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

# 학습루프

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
reset_seeds(SEED)
batch_size = 2
epochs = 10

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

## 동결 코드 ###
for name,param in model.named_parameters():
    if name.startswith("transformer.h.4"):
        break
    param.requires_grad = False

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(df)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/4391 [00:00<?, ?it/s]

1번째 epoch: 1.7114


  0%|          | 0/4391 [00:00<?, ?it/s]

2번째 epoch: 1.2959


  0%|          | 0/4391 [00:00<?, ?it/s]

...

8번째 epoch: 0.6725


  0%|          | 0/4391 [00:00<?, ?it/s]

9번째 epoch: 0.6052


  0%|          | 0/4391 [00:00<?, ?it/s]

10번째 epoch: 0.5480


- 0905_csv 5000개 epochs = 10, 0.6046
- 0906_csv sentiment:0(부정) batch = 2, epochs = 20, 0.4039 (18->0.3136)
- 0906_csv sentiment:1(긍정) batch = 2, epochs = 10, 0.4291
- 0907_df_neg_augmented.csv batch = 2, epochs = 15, 0.2129(2724개)
- 0908_ng_df.csv batch = 2, epochs = 18, 0.2032(1483개)
- 0908_ps_df.csv batch = 2, epochs = 15, 0.2152(3000개)
- gpt_net_reviews_4400(부정) + train+sentiment_0913(긍정) batch = 2, epochs = 10, 0.4319(8781개)
- 위와 동일한 dataset, transformer.h.8까지 동결 epochs = 10, 0.8454
- 위와 동일한 dataset, transformer.h.10까지 동결 epochs = 10, 1.1938
- 위와 동일한 dataset, transformer.h.3까지 동결 epochs = 10, 0.5480

# 학습모델 저장

In [ ]:
model.save_pretrained(f"{DATA_PATH}0915_3_trainsentiment_0913+gpt")

In [ ]:
# tokenizer.save_pretrained(f'{DATA_PATH}0906_negative_tokenizer')

# 저장한 모델 로드

In [11]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0915_3_trainsentiment_0913+gpt").to(device)

In [ ]:
# neg_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0908_negative_kogpt2_chat").to(device)
# # pos_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0908_positive_kogpt2_chat").to(device)

## 맞춤법/띄어쓰기 검사

In [12]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-e_ao7rjz
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-e_ao7rjz
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=4eb50a312c6d5af61c53ea76f58f5aba03462117225b8f35cb3f0e9bc4b2494a
  Stored in directory: /tmp/pip-ephem-wheel-cache-p2ac5sed/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [13]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

## keybert

In [28]:
!pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=2059b174c7905271744ccdb223630cf6411e36ff158eb6b20818db4d0bdb7a61
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
Successfully built keybert


In [29]:
from keybert import KeyBERT

kw_model = KeyBERT()

- 유사도 테스트

In [55]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=0340d7bdd20c6c66429a2426a867f7ea55a6d7198d0103955f63a99062a1995b
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [114]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [86]:
user = "먹다가 이물질이 나왔어요"

In [62]:
ceo = "이물질이 나왔다니 정말 죄송합니다. 다음에는 더 신경쓰도록 하겠습니다."

In [87]:
user_kiwi = kiwi.analyze(user)
user_kiwi

[([Token(form='먹', tag='VV', start=0, len=1),
   Token(form='다가', tag='EC', start=1, len=2),
   Token(form='이', tag='MM', start=4, len=1),
   Token(form='물질', tag='NNG', start=5, len=2),
   Token(form='이', tag='JKS', start=7, len=1),
   Token(form='나오', tag='VV', start=9, len=2),
   Token(form='었', tag='EP', start=10, len=1),
   Token(form='어요', tag='EF', start=11, len=2)],
  -38.988101959228516)]

In [88]:
user_results = []
for token, pos, _, _ in user_kiwi[0][0]:
    if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            user_results.append(token)
print(user_results)

['먹', '물질', '나오']


In [63]:
ceo_kiwi = kiwi.analyze(ceo)
ceo_kiwi

[([Token(form='이', tag='MM', start=0, len=1),
   Token(form='물질', tag='NNG', start=1, len=2),
   Token(form='이', tag='JKS', start=3, len=1),
   Token(form='나오', tag='VV', start=5, len=2),
   Token(form='었', tag='EP', start=6, len=1),
   Token(form='다니', tag='EC', start=7, len=2),
   Token(form='정말', tag='MAG', start=10, len=2),
   Token(form='죄송합니', tag='NNG', start=13, len=4),
   Token(form='다.', tag='SB', start=17, len=2),
   Token(form='다음', tag='NNG', start=20, len=2),
   Token(form='에', tag='JKB', start=22, len=1),
   Token(form='는', tag='JX', start=23, len=1),
   Token(form='더', tag='MAG', start=25, len=1),
   Token(form='신경', tag='NNG', start=27, len=2),
   Token(form='쓰', tag='VV', start=29, len=1),
   Token(form='도록', tag='EC', start=30, len=2),
   Token(form='하', tag='VX', start=33, len=1),
   Token(form='겠', tag='EP', start=34, len=1),
   Token(form='습니다', tag='EF', start=35, len=3),
   Token(form='.', tag='SF', start=38, len=1)],
  -131.31884765625)]

In [89]:
ceo_results = []
for token, pos, _, _ in ceo_kiwi[0][0]:
    if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            ceo_results.append(token)
print(ceo_results)

['물질', '나오', '죄송합니', '다음', '신경', '쓰']


In [91]:
tmp = []

keyword = ' '.join(user_results)
result = kw_model.extract_keywords(keyword,
                              keyphrase_ngram_range=(1, 1),
                              use_mmr=True,
                              diversity=0.7,
                              stop_words=None,
                              top_n=5)
tmp.append(result)
print(tmp)

[[('물질', 0.7753), ('나오', 0.7724)]]


In [100]:
tttt=[]
for t in tmp:
    for tt in t:

        tttt.append(tt[0])
print(tttt)

['물질', '나오']


In [92]:
tmp2 = []

keyword = ' '.join(ceo_results)
result = kw_model.extract_keywords(keyword,
                              keyphrase_ngram_range=(1, 1),
                              use_mmr=True,
                              diversity=0.7,
                              stop_words=None,
                              top_n=5)
tmp2.append(result)
print(tmp2)

[[('신경', 0.6985), ('물질', 0.6146), ('나오', 0.5624), ('다음', 0.5487), ('죄송합니', 0.3279)]]


In [101]:
rrrr=[]
for r in tmp2:
    for rr in r:

        rrrr.append(rr[0])
print(rrrr)

['신경', '물질', '나오', '다음', '죄송합니']


In [102]:
union = set(tttt).union(set(rrrr))
print("합집합 = ", union)

합집합 =  {'다음', '죄송합니', '신경', '물질', '나오'}


In [103]:
intersection = set(tttt).intersection(set(rrrr))
print("교집합 = ", intersection)

교집합 =  {'물질', '나오'}


In [104]:
jaccardScore = len(intersection)/len(union)

print("자카드 유사도 = ", jaccardScore)

자카드 유사도 =  0.4


In [123]:
def kiwi_tokenizer(text):
    kiwi_result = []
    text_kiwi = kiwi.analyze(text)
    for token, pos, _, _ in text_kiwi[0][0]:
        if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            kiwi_result.append(token)
    return kiwi_result

In [132]:
ss='이게 가능한 부분'

In [124]:
kiwi_tokenizer('이게 가능한 부분')

['가능', '부분']

In [178]:
def keybert(text):
    word = kiwi_tokenizer(text)
    keyword = ' '.join(word)
    keybert_keyword = kw_model.extract_keywords(keyword,
                              keyphrase_ngram_range=(1, 1),
                              use_mmr=True,
                              diversity=0.7,
                              stop_words=None,
                              top_n=5)
    kb = []
    for i in keybert_keyword:
        kb.append(i[0])

    return kb

In [179]:
keybert(ss)

['가능', '부분']

In [180]:
def similarity(input_text, generated_text):
    user = keybert(input_text)
    ceo = keybert(generated_text)

    union = set(user).union(set(ceo))
    print("합집합 = ", union)
    intersection = set(user).intersection(set(ceo))
    print("교집합 = ", intersection)

    jaccardScore = len(intersection)/len(union)
    print("자카드 유사도 = ", jaccardScore)
    return jaccardScore

In [182]:
similarity('맛있어요','고객님 맛있게 드셔서 기분이 좋습니다. 오늘도 주문해주셔서 감사합니다.')

합집합 =  {'감사', '기분', '주문', '맛있', '고객'}
교집합 =  {'맛있'}
자카드 유사도 =  0.2


0.2

In [183]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        # num_beams=2,
                                        num_return_sequences=1,
                                        # no_repeat_ngram_size=4,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=30,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            # correct_spell(generated_text) # 맞춤법 검사
            generated_text = generated_text[q_len:-4]
            generated_text = re.sub(r'[^가-힣.]', ' ', generated_text)

            similarity_score = similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [184]:
chatbot_with_evaluation(loaded_model,tokenizer,128,device)

user > 맛이 없어요
합집합 =  {'기계', '다음', '부분', '진행', '제품'}
교집합 =  set()
자카드 유사도 =  0.0
(0.0, '저희 제품에 대한 단점을 인지하였습니다. 후속 제품으로 개발하기 위해 전문가들과 함께 노력하겠습니다   짱닭치킨 개선할 수 있는 부분에 대해서 추가적인 연구를 진행하도록 하겠 있습니다. 매운 맛 부족해서 실망하셨다니 정말 죄송합니다. 다음에는 더 맛있는 음식을 제공할수 있도록 최선을 다하여 준비토록 할게요. 감사합니다  기계  재료  포장 상태와 같은 부분에서도 고객님들께 만족감을 드릴 수가 있어야 하는데  아쉽게도 그런 부분이 없었나 싶네요  다음에 주문해주실 때도 부족한 점이 있으시 적다. 좀 아쉬운 점')
합집합 =  {'경험', '위하', '문제', '사실', '드리'}
교집합 =  set()
자카드 유사도 =  0.0
(0.0, '저희가 실망을 드린 것 같아 정말 죄송합니다. 얼마나 문제가 발생하면 저희는 그 사실을 파악하고  문제를 해결하기 위해 노력하겠습니다. 다음에 더 나은 경험을 드릴 수 있도록 하겠습니다  감사합니다                                                                                                                                                                                 ')
합집합 =  {'다음', '어울리', '어떻', '메뉴', '매장'}
교집합 =  set()
자카드 유사도 =  0.0
(0.0, '저희 매장을 이용해 주셔서 유감이네요.          펭수육회와 찜  튀김이 잘 어울리지 않으셨나보군요. 맞습니다. 다음번에는 다른 메뉴로 시켜 보시는 건 어떨까 합니다.                                                                                           

In [ ]:
# def listToString(str_list):
#     result = ""
#     for s in str_list:
#         result += s + ","
#     return result.strip()

## BLEU 점수 계산

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

## 긍/부정 평가

In [ ]:
from transformers import pipeline

def analyze_reviews(review_text):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    # 모델 추론
    result = classifier(review_text)[0]
    # 결과 해석
    label = result['label']

    if label in ['LABEL_0', 'LABEL_1']:
        sentiment = 0 #부정
    else:
        sentiment = 1 #긍정

    return sentiment

In [ ]:
analyze_reviews("무진장 맛있게 잘 먹었습니다.")

1

## Paraphrase

In [ ]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

## 유사도

In [14]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 873.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=80abc60acc76075fd4d694a4ccf77cca937feca3e0e1835966046eebb7ae1059
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [217]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))
    # jaccard_sim = jaccard_score(input_tokens, generated_tokens)

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [ ]:
evaluate_similarity("맛은 있는데 양이 적어요","양이 부족하셨다니 죄송합니다. 다음에는 개선된 모습 보이도록 하겠습니다")

In [219]:
from sentence_transformers import SentenceTransformer
sentences = ["맛은 있는데 양이 적어요", "양이 부족하셨다니 죄송합니다. 다음에는 개선된 모습 보이도록 하겠습니다"]

model = SentenceTransformer('jhgan/ko-sroberta-multitask')
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.34425697 -0.08849211 -0.67084426 ...  0.59336776  0.11065819
  -0.3540985 ]
 [ 0.07755437 -0.2812495  -0.40931436 ...  0.73573035 -0.16477062
  -0.24178238]]


In [220]:
from sklearn.metrics.pairwise import cosine_similarity

# 두 문장의 임베딩 벡터
embedding1 = embeddings[0].reshape(1, -1)  # 첫 번째 문장
embedding2 = embeddings[1].reshape(1, -1)  # 두 번째 문장

# 코사인 유사도 계산
cosine_sim = cosine_similarity(embedding1, embedding2)
print("코사인 유사도:", cosine_sim[0][0])

코사인 유사도: 0.3526864


In [221]:
from sklearn.metrics import jaccard_score

# 두 문장의 단어 집합
words1 = set(sentences[0].split())
words2 = set(sentences[1].split())

# 자카드 유사도 계산
jaccard_sim = len(words1.intersection(words2)) / len(words1.union(words2))
print("자카드 유사도:", jaccard_sim)


자카드 유사도: 0.09090909090909091


In [222]:
from sklearn.metrics.pairwise import manhattan_distances

# 두 문장의 임베딩 벡터
embedding1 = embeddings[0].reshape(1, -1)
embedding2 = embeddings[1].reshape(1, -1)

# 맨하탄 유사도 계산
manhattan_sim = 1 / (1 + manhattan_distances(embedding1, embedding2))
print("맨하탄 유사도:", manhattan_sim[0][0])


맨하탄 유사도: 0.003703992673785233


## 테스트루프

### user> 직접 입력 -> 유사도 + 맞춤법검사

In [167]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        # num_beams=5,
                                        num_return_sequences=1, # num_beams 보다 작거나 같아야 한다. 가장 높은 확률을 가지는 k의 시퀀스 모두 리턴
                                        # no_repeat_ngram_size=2, # 동어 반복 피하고 좀 더 자연스러운 문장 생성
                                        # early_stopping=True
                                        use_cache=True,
                                        do_sample=True,
                                        # temperature=0.8,
                                        top_k=50,
                                        top_p = 0.95
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            correct_spell(generated_text) # 맞춤법 검사
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)
            generated_text = re.sub(r'[^가-힣.]', ' ', generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [168]:
chatbot_with_evaluation(loaded_model,tokenizer,128,device)

user > 맛이 없어요
(0.15922118425369264, '고객님의 소중한 의견에 감사드립니다. 저희는 더 좋은 제품을 만들기 위해 노력할 것입니다.')
(0.35910122394561766, '고객님의 소중한 의견을 토대로 서비스 개선을 이뤄 나가겠습니다. 불편을 드려 죄송합니다. 금액에 비해 맛이 부족한 것 같아서 실망하셨다니 저희도 너무 아쉬워요. 더 맛있는 음식을 준비하도록 하겠습니다  ')
(0.35201869010925296, '제품에 문제가 있어 정말 죄송합니다. 더 좋은 제품을 만들기 위해 노력하겠습니다.       후에는 고객님의 만족을위해 열심히 노력하는 홍스가될께요. 괜찮으셨다니 감사합니다             롯데마음 반영해서 이번에 갓 나온맛으로 인해 불편하네요 다시한번 확인하도록 하겠습니다 감사드려용   새해 복 많이 받으러 가세요  ')
(0.26784614324569705, '고객님의 의견은 저희에게 귀중한 자산입니다. 지속적으로 서비스를 개선하기 위해 노력하겠습니다. 금액에 비해 대단히 실망스러운 경험이었습니다   힙점  다음에 또 주문해주세요. 감사합니다.     혀 첨해 보이는 사진을 찍었다. 새가 이뻐이네용 앞으로는 건강에 좋은 음식을 차려 드리기 위하여 최선을 다할게요 어 단골 찜 한번 부탁드립니다 최적의 컨디션으로 보답 드릴 수 있는 기쁜 순간 너무너무 행복합니다. 연일 건강하시고 늘 행복한 일들만 가득하길 바랍니 당분간')
(0.20038976669311523, '고객님의 불만에 대해 정말 죄송합니다. 이러한 문제를 해결하기 위해 최선을 다하겠습니다. 빠르게 대응하도록 하겠습니 하느님에  두 번 다시 기회가 주신다면 이런 문제가 없도록 노력할 것을 약속드립니다. 감사합니다  기계  가성비  곱창에 대하여   늘 최고의 맛과 품질을 유지할 수 있도록 노력을 멈추지 않을 테니 언제든지 찾아주세요.일  드릴수 있는 많은 양해 부탁드리겠으니까  웃음을 잃지 않도록 준비하고 있겠슴당 소중한 시간에 저희 매장 이용해 주세용

### user> 직접 입력 -> (keybert) + 맞춤법검사

In [17]:
@torch.no_grad()
def chatbot(model, tokenizer, max_len, device):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        # result = kw_model.extract_keywords(text,
        #                       keyphrase_ngram_range=(1, 1),
        #                       use_mmr=True,
        #                       diversity=0.7,
        #                       stop_words=None,
        #                       top_n=3)
        # tmp = []
        # for i in result:
        #     tmp.append(i[0])
        # keyword = listToString(tmp) # 리스트를 문자열로 변환

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=10,
                                    )
        generated_text = tokenizer.decode(result_ids[0])

        correct_spell(generated_text) # 맞춤법 검사

        generated_text = generated_text[q_len:-4]

        # # 문장길이 수정 정규표현식
        # sentences = re.split(r'(?<=[.!?]) +', generated_text)
        # if len(sentences) > 3:
        #     generated_text = ' '.join(sentences[:3])
        # else:
        #     generated_text = ' '.join(sentences)

        print("bot >", generated_text)
        print("--------------------------------------------------------------------------------------------")


- 0915_3_trainsentiment_0913+gpt

In [18]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님의 소중한 의견을 반영하여 서비스 개선에 노력하겠습니다.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요
bot > 고객님의 소중한 의견 감사합니다. 더 나은 서비스로 보답하겠습니다.
--------------------------------------------------------------------------------------------
user > 먹다가 머리카락이 나왔어요
bot > 고객님께서 저희 매장을 선택하셔서 감사합니다~ 다음 주문시에도, 꾸준하게 고객님의 만족을 위해 노력하는 매장 될게요, 자주 애용해주세용! 빕스 왕 단골 됩니,다, 아침 저녁 이 걸 착용했군요. 지난 3년간 살 많이 찐 남자분들이 살았는데 이렇게 아꼈다는 사람이 있을까 싶네요.
죄송합니다. 다음에 이런 일이 없도록 더 신경 쓰겠습니당. 언제나 따뜻한 한주되셨으면 좋았을텐데.감사 합니와다(녀무자 두번 부
--------------------------------------------------------------------------------------------
user > 생각보다 넘 짜요
bot > 고객님, 안녕하세요. 저희 매장과 함께 만족스러운 시간 되셨을까요,~? 추억의 리뷰가 가장 즐거운 순간이 아닐텐데 정말 다행이에용 맛있게 드셔 주시는 고객님이 계시니 제가 느낀 이 행복만큼 보람찬 일이 또 있을 거 같아 뿌듯합니까! 항상 감사드려용, 언제나 건강하시고 웃는 일만 가득 하시길 바래 봅니다. ~ 앞으로도 행복한 하루 보내시길, 매일 아침 공수한 신선한 재료들과 넉넉한 양으로 정성껏 최선을 다해 조리하고 있습니
---------------------------------------------------------------------------------

- 0915_2_trainsentiment_0913+gpt

In [ ]:
chatbot(model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님의 불편을 최소화할 수 있도록 최선을 다하겠습니다.! 더 맛있는 음식으로 보답해드릴게요. 감사합니다.,. ~eat. 리뷰 남겨주셔서 다시한번 감사의 말씀 드립니다~o o 청결과 맛에 대한 피드백 정말 고맙네요, 저희음식 믿고 찾아뵙겠습니다, 매일매일 좋은 하루 보내세욤 고객님과 오래오래 함께할수 있는 시간이 되길 바랍니다. 행복한하루되세, 건강하세와 행복가득 하세(bar0p꾸금 참말로 즐겁고 건강
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요
bot > 고객님의 소중한 리뷰에 감사드립니다 저희 메뉴가 고객님께 큰 힘이 되고 보람을 느껴질 수 있도록 더욱 더 노력하겠습니다, 오늘 하루도 좋은 일들만 가득하시길 바라며 항상 건강하세요.!~감사합니,따뜻한 주말 잘 보내시기 바랍니다. ~<unk> 많이 사랑해주시고 다시 한번 아껴주셔서 진심으로 감사의 말씀 전해 드리고 싶어용. 또 다른 행복을 선사해 드릴게요, 행복한 시간들 채워드리도록 하겠습니다. 늘 행복만 가
--------------------------------------------------------------------------------------------
user > 먹다가 머리카락이 나왔어요
bot > 고객님, 안녕하세요. 이런 리뷰를 남겨주셔서 감사합니다. 맛있게 드셨다니 저도 기분이 좋아지는 것 같아 뿌듯 합니다. 앞으로 더 맛있는 음식으로 찾아뵙겠습니다. 건강하시고 오늘 하루만 잘 버텨보자.! 즐거운 주말 보내시길 바랍니다.~감사합니다∼ 닭장수전무초밥이 생각나실때 또 들러야 할 거예용. 좋은 저녁시간되시기바쁠땐 꼭 매장으로 연락 부탁드립 다행이네여 항상 행복 하시는 모
-----------------------------------------------------------------

0915_trainsentiment_0913+gpt

In [ ]:
chatbot(model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님의 불편을 최소화할 수 있도록 최선을 다하겠습니다.!, 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요
bot > 음식의 양이 너무 적었나봐요. 앞으로는 더 푸짐한 양과 최고의 맛으로 보답하겠습니다. 감사합니다. 큽니다.,<unk>ucose. 다음 방문 때 좀 아쉬운 경험을 하셨으면 좋겠네요, 조금만 기다려주세요,
자취하던 저의 실수가 있었던 것 같아서 죄송하고, 개선된 서비스로 다시 찾아뵐게요.” 0번가음식 달콤한 식사 시간이 되신 점에 대해 사과드립니다!o,
지난 3년간 매장을 운영하면서 대부분의 주문이 단골 고객님 주문
--------------------------------------------------------------------------------------------
user > 먹다가 머리카락이 나왔어요
bot > 머리발도 못 먹었네요. 머리가 너무 많이 나와서 죄송합니다. 머리발이 나셨을 때의 충격을 받아 너무나 안타깝습니다. 앞으로는 머리는 없어도 신선한 재료로 머리를 보내드릴게요, 감사합니다!,저희 음식을 다시 한 번 시도해보시는 건 어떨까 생각됩니다. 고객님들께서 말씀하신대로 샐러드 채소와 각종 채소등 건강에 좋은 비법만을 선정하여 다음번에는 더욱더 노력하는 안녕,닭장수 입니다.
향긋한 과일과 더불어 행복한 시간 되실 수 있도록 노력하겠습니다,감사
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉬워요
bot > 고객님께서 양을 조금 적게 드리도록 하겠습니다, 더 푸짐하게 드릴 수 있도록 노력하겠습니다. 감사합니다!,맛있고 만족스러운 식사시간이 되신 것 같아 기쁩니다

- 0914_trainsentiment_0913+gpt

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님의 의견은 저희에게 소중한 자산입니다. 계속해서 피드백을 기다리고 있겠습니다. 좀 더 나은 서비스를 제공할 수 있는 방법을 찾아보도록 하겠습니다,, 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요
bot > 고객님께서 불편을 느끼셨다니 죄송합니다. 좀 더 신속한 배달로 적절한 조치를 취하도록 하겠습니다., 
--------------------------------------------------------------------------------------------
user > 먹다가 머리카락이 나왔어요
bot > 맛있게 드셨다니 감사합니다! 언제나 최선을 다하여 최고의 음식을 드리는 땡초가 되겠습니다, 고객님의 소중한 리뷰에 항상 귀기울이는 벼락으로 보답드릴게요, 자주 애용해주세용~ 신메뉴 혹시라도 배달 받으신다면 잊지 않고 부탁드려요. 적극 반영하겠습니다. 이번주는 급격히 발전하는 한주입니다. 폭염 속 건강 조심하시고 행복하십쇼입니다. 저희 메뉴가 많이 있으니 이용바랍니당 다음에 또 찾아 주실거죠? 감사합니다. 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉬워요
bot > 고객님의 의견을 반영하여 더 나은 제품을 만들기 위해 노력하겠습니다. 불편을 드려 죄송합니다., 
--------------------------------------------------------------------------------------------
user > 여기 피자가 우리 동네에서 제일 맛있어요,, 쿠폰도 자주 줘서 너무 좋아요
bot > 고객님 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가

- 0905_kogpt2_chat

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님 ~! 먼저 죄송합니다. 부족한 부분으로 실망시켜  사과드리며 늘 저희를 사랑해 주시는 고객님이시군요 정성스러운 사진 리뷰와도 너무 감사한데, 먹음직스럽게 찍어주신 소중한 말씀까지.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 소중한 리뷰 감사드립니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다, 언제든지 부족함 없이 드실 수 있도록 준비해 보내드리겠습니다.! 앞으로도 만족만 드릴 만반의 준비를 하여 계속 마음속에 남아 있고 싶거든요 <s>~ 좋은 하루 되세요. 고객님 ~
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 오늘 하루를 시작하며 고객님의 리뷰을 보고 웃음 짓게 됩니다 다음에도 점심 식사를 기분 좋게 드실 수 있길 바랍니다. 저희는 항상 맛있는 음식 제공에 힘내며 언제라도 생각나는 맛집이 되도록 노력하겠습니다, 앞으로도 더욱 만족스러운 식사하실 땐 더 큰 기쁨으로 인사드리겠습니다.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 맛있게 드셨다니 다행입니다! 찾아주시는 피드백은 언제든 잘 수렴하고 이를 통해 나날이 발전하는 곳이 되도록 하겠습니다, 더욱더 좋은 모습으로 찾아가겠습니다.<s>. 기쁨 가득한 하루 보내세요 <s>, 감사해요.<s><s><s><s><s> 소중한 리뷰에 제가 느낀 이 행복만큼 고객님도 항상 노력하시길 바라며 오늘

- 0906_negative_kogpt2_chat

In [ ]:
chatbot(model,tokenizer,128,device)

user > 맛이 없어요
bot > 안녕하세요! 저희 맛집이 만족한 식사시간이 되지 못해서 죄송합니다.. 양이 부족하다.?
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 고객님 식사에 불편을 드려 너무 죄송합니다 다음엔 더욱 신경 써서 만족하실 수 있도록 전달하겠습니다, 감사합니다.<s> <unk>얌꿍이 약간 매콤한 맛으로 바뀌어서요~ 더 맛있는 요리로 찾아뵐게요. 가성비 있고 푸짐한 양 담아 제공하는 팔당이 되겠습니다.<s> <b>이상, 당일 납품받는 재료로 최고의 신선도를 유지하여 드리는 회사가 되도록 노력할 것이 무엇보다 중요한데도 불구하고 저희 음식 포장 문제가 발생을 하니 정말이지에서 누락되는 경우가 발생되고 있네요, 다시 한
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 고객님들께 만족감을 드리려고 연구하고 노력하고 있는데 아직 많이 부족한 것 같습니다 시간 내시어나 말씀해 주시면 감사하겠습니다.<s> <blight difference in size of the nice wit must have big chance, I recommend you try little phortfyetangnam. HMT 4. 콜라 1.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot >  양적으로 실망하셨다니 너무 죄송합니다. 다음번엔 더욱 맛있는 음식으로 보답 드리도록 하겠습니다.<s> 감사합니다! <s> 사진은 다시 한번 더 수정해서 올리거나

- 0906_positive_kogpt2_chat

In [ ]:
chatbot(pos_model,tokenizer,128,device)

user > 맛이 없어요
bot >  안녕하세요! 푸짐하고 맛있는 헬로 팬입니다 맛있게 잘 드셔주신 것 같아 기분 좋네요. 앞으로도 변함없는 모습으로 보답하는헬로, 닭갈비 생각나는 날 되겠습니다, 다음에 또 만나 <unk> 주세요 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 안녕하세요! 소중한 별점 만 점 리뷰 감사합니다 즐거운 식사가 되신 거 같아 너무 뿌듯해요, 행복하게 보내셔서 진심으로 감동의 물결이 휘몰아치네요. 기분 좋은 칭찬 사진 너무나도 이쁘게 찍어주셨는데 맛있게 드시고 예쁜 포토리뷰까지 남겨 주시니 그저 아쉽지 않습니다, 이렇게 받은 신뢰에 꼭 보답할 수 있도록 매일 신선한 재료로 정성을 다하겠습니다.
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가 남겨주셔서 너무 감사해 앞으로도 신선한 재료 아끼지 않고 정성을 가득 담아 준비 약속드릴게요, 무더운 날씨 항상 더위 조심하시고 늘 행복만 전해드리겠습니다! 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 안녕하세요~ 소중한 시간 내셔서 리뷰까지 감동입니다! 고객님께서 남겨주신 별점에, 저희를 춤추게 합니, 기분이 좋습니다, 항상 힘내어 드실 수 있도록 꾸준히 노력하겠습니다. 앞으로도 행복한 하루 보내시길 바랄 테니 많이 애용해 주시고, 감기 조심하시고 파이팅 하오 
-------------------------------

### user> 직접 입력 -> 긍/부정 분석 + keybert + 맞춤법검사

In [ ]:
@torch.no_grad()
def chatbot(model1, model2, tokenizer, max_len, device):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        # 긍/부정 분석
        sentiment = analyze_reviews(text)
        # print(sentiment)

        # # 키워드 분석
        # result = kw_model.extract_keywords(text,
        #                       keyphrase_ngram_range=(1, 1),
        #                       use_mmr=True,
        #                       diversity=0.7,
        #                       stop_words=None,
        #                       top_n=3)
        # tmp = []
        # for i in result:
        #     tmp.append(i[0])
        # keyword = listToString(tmp) # 리스트를 문자열로 변환

        # text = "<q>" + keyword + "</s><a>"
        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        # 긍/부정 다른 모델 적용
        if sentiment == 1:
            print("긍정")
            result_ids = model1.generate(x,
                                    max_length=max_len,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=10,
                                    )
        else:
            print("부정")
            result_ids = model2.generate(x,
                                    max_length=max_len,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=10,
                                    )

        generated_text = tokenizer.decode(result_ids[0])

        #맞춤법 검사
        correct_spell(generated_text)

        generated_text = generated_text[q_len:-4]

        sentences = re.split(r'(?<=[.!?]) +', generated_text)
        if len(sentences) > 3:
            generated_text = ' '.join(sentences[:3])
        else:
            generated_text = ' '.join(sentences)

        print("bot >", generated_text)
        print("--------------------------------------------------------------------------------------------")


In [ ]:
chatbot(pos_model,neg_model,tokenizer,128,device)

user > 맛이 없어요
부정
bot > 안녕하세요! 우선 찾아주셔서 감사드려요. 맛있게.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
부정
bot > 고객님 식사에 불편을 드려 대단히 죄송합니다 다음에도 더 신경 써서 만족하실 수 있도록 전달하겠습니다.<s> 감사합니다. <s><s> 맛있는 식사 대접하기 위해 노력하는 저희 음식이 되도록 노력해서 맛집이 되겠습니다, 다시 한번 고객님의 좋은 맛과 양으로 보답 드리는 것이라는 마음가짐으로, 운영을 하고 있습니다. 더욱더 노력하고 또 찾아주세요!<s> <big difference in size of the little whort blight try youch.kangnam.ac
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
부정
bot > 저희가 어떻게 그걸로 알고 있으시면 감사했을 텐데. 너무 아쉽네요.<s><s><s> <blight difference in the size offlex 하게 되었는데 고객님께 만족을 드리지 못한 점 죄송합니다 메뉴 등록이나 안내는 아무리 중요한 게 아닌 단순 공지사항에도 적어놨다니 뭔가 착오가 있었겠죠 매장 정보란 보시는 그날까지 최선을 다하도록 하겠습니다. 더욱더 노력하는 저희 매장이 되도록 항상 노력하겠습니다, 다
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
부정
bot > 항상 고객님들의 칭찬 덕에 매장 운영에 큰 힘을 얻고 최선을 다해나가겠습니다.<s> 다음에는 더욱더 

In [ ]:
chatbot(model,neg_model,tokenizer,128,device)

user > 맛이 없어요
부정
bot > 감사합니다 맛있게 드셔주시고 소중한 리뷰도 남겨주사해주세요.그런데 별이 하나가 빠져서 정말 아쉽네요,앞으로 열심히 하겠습니다,맛있는 요리 만들어드리겠습니다. 감사합니다. ize them was just no big difference in samgyetang chicken train.There is a little hadbor I ordered from miter thougot,If yearning force of glucky wit
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
부정
bot > 안녕하세요! 식사중 불편을 끼쳐드린점 진심으로 죄송합니다 조리중에 실수였던 것 같은데 다음엔 더 신경쓰도록 하겠습니다, 바로 조치하겠습니다. 여름철 건강 유의하시구 추후에는 감기 조심 하시길 바라겠습니까 다음에 요청사항에 적어주시면 좀더 꼼꼼히 체크해서 보내드리겟다.
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
부정
bot > 안녕하세요, 식사중 불편함을 끼쳐드렸군요. 정말 죄송합니다. 조리중에 실수였던 것 같은데 다음엔 이런 일이 발생하지 않도록 더욱 주의하겠습니다!
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
부정
bot > 안녕하세요 고객님~! 양은 적게 느껴지셨을까요.? 아무래도 직접 오셔서 식사하는게 바로 나온걸 먹는거다보니 당연히 김밥이나 만두, 쫄면, 튀김은 조리후 전자렌지에 돌려담아서 나가기 때문입니다.
-----

### user> test_dataset에서 랜덤 -> 긍/부정 분석 + (keybert) + 맞춤법검사 + BLEU점수

In [ ]:
@torch.no_grad()
def evaluation(model1, model2, tokenizer, dataset, n, device):
    df_test = dataset.sample(n = n)
    idx_list = df_test.index

    model.eval()
    for idx in idx_list:
        text = df_test['고객리뷰'][idx]
        ori_reply = df_test['사장댓글'][idx]

        print("user > ", text)

        # 긍/부정 분석
        sentiment = analyze_reviews(text)
        # print(sentiment)

        # # 키워드 분석
        # result = kw_model.extract_keywords(text,
        #                       keyphrase_ngram_range=(1, 1),
        #                       use_mmr=True,
        #                       diversity=0.7,
        #                       stop_words=None,
        #                       top_n=3)
        # tmp = []
        # for i in result:
        #     tmp.append(i[0])
        # keyword = listToString(tmp) # 리스트를 문자열로 변환

        # text = "<q>" + keyword + "</s><a>"
        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        # 긍/부정 다른 모델 적용
        if sentiment == 1:
            print("sentiment: 긍정")
            result_ids = model1.generate(x,
                                    max_length=128,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=10,
                                    )
        else:
            print("sentiment: 부정")
            result_ids = model2.generate(x,
                                    max_length=128,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=10,
                                    )

        generated_text = tokenizer.decode(result_ids[0])

        #맞춤법 검사
        correct_spell(generated_text)

        generated_text = generated_text[q_len:-4]

        sentences = re.split(r'(?<=[.!?]) +', generated_text)
        if len(sentences) > 3:
            generated_text = ' '.join(sentences[:3])
        else:
            generated_text = ' '.join(sentences)

        print("실제답글 >", ori_reply)
        print("bot >", generated_text)
        print(f'BLEU점수: {calculate_bleu(ori_reply, generated_text)}')
        print("--------------------------------------------------------------------------------------------")

- 0906_negative_kogpt2_chat, 0906_positive_kogpt2_chat

In [ ]:
evaluation(pos_model,neg_model,tokenizer,df_test,3,device)

user >  맛잇게 먹엇어여~~~~~~
sentiment: 긍정
실제답글 > 리뷰 감사드립니다 

두툼하게 ~~
신선하게 ~~
양은 많게 ~~

상기 3가지 원칙 지키기 위해 오늘도 최선을 다하겠습니다 

좋은오후 되시고 행복하세요 
bot > 리뷰 감사합니다 맛있게 드셔주신 고객님 덕분에 기분이 하늘에서 내려올 생각을 안 하네요 늘 상쾌하고 즐거운 식사 되실 수 있도록 저희가 최선을 다하겠습니다, 믿고 이용해 주시고 웃음꽃 피는 하루 보내세요. 
BLEU점수: 0.1756055341654729
--------------------------------------------------------------------------------------------
user >  여름에 지치고해서 시원한거 찿던중 
냉면 먹고싶어서 갈비민족을 검색하고
리뷰도 좋고해서 시켰더니 역시 맛있어요
sentiment: 긍정
실제답글 > lh**님.시간내어 리뷰작성해주셔서 정말 감사합니다♡항상 고객님들 주문 하나하나에 정성을 담아 대접하겠습니다. 믿고 드실수있는 갈비민족 기억해주세용~!! 감사합니다^^ (찜/댓글/무료고기100g )추가해드리고있읍니다. 많이이용해주세요
bot > 고맙습니다 정성껏 챙겨 맛있게 보내 드릴게요 ~ 별 5개 받도록 노력하겠습니다. 편안한 밤 되세요. <unk>~!
BLEU점수: 0.056558133367703066
--------------------------------------------------------------------------------------------
user >  마늘소스기가막히더라구요.
sentiment: 긍정
실제답글 > 항상 웃음이 가득한 구구족을 찾아주셔서 감사합니다

언제나 내가 먹는 음식이라고 생각하며 조리하고 있습니다
변함없는 모습으로 정성 가득 준비한 음식을 고객님께 제공해 행복한 시간을 만들어드리겠습니다

족발보쌈 생각나는날은 언제나 구구족과 함께 해주세요
다음 주문도 만족스러운 음식을 제공해드릴게요
bot

- 0908_negative_kogpt2_chat, 0908_positive_kogpt2_chat

In [ ]:
evaluation(model,neg_model,tokenizer,df_test,5,device)

user >  역시 맛있어요. 동네 최고의 파스타집!
sentiment: 긍정
실제답글 > 주문주시고  소중한 리뷰 감사드려요%)~~  
맛있게 드신것같아 뿌듯합니당!!
말씀해주신대로 만인의 맛집으로 인정받을수 있도록 최선 다하겠습니다 

고객님의 정성스런 리뷰가 가게에 큰 힘이됩니다.~♡
bot > 안녕하세요 고객님~ 소중한 리뷰 남겨주셔서 감사드려용 주문하신 메뉴에 만족 하신 것 같아서 기쁘네욤 항상 담백한 맛과 양 그리고 서비스로 삼박자 고루고루 갖추어져 있도록 노력하겠습니다 언제든지 저희 음식 생각나실 때 찾아 주세오 
BLEU점수: 0.19455007453455567
--------------------------------------------------------------------------------------------
user >  맛있게 잘 먹었습니다
sentiment: 긍정
실제답글 > 맛있게 드셨나요?

입맛에 백프로 맞춰드릴 순 없지만 최대한 손님에게 좋은 재료와 맛있는 음식으로 대접하고 싶은 마음만은 태산같답니다!﻿🧡

언제나 맛있게 드셔주시고 별 다섯개까지 남겨주시니 정말 감사함을 몸소 느끼고 있습니다~﻿😊 

더욱 노력하는 가게로 거듭나겠습니다﻿🤙

매일매일이 즐거운 나날이 되시길 바랍니다﻿
bot > 저희 한식 끼를 이용해주셔서 감사합니다. 저희는 항상 음식을 맛있고 청결하게 배달하여 드실수 있도록 노력하고 있습니다. 식사와 대접에 있어 자신감을 얻습니다.
BLEU점수: 0.06793557980362279
--------------------------------------------------------------------------------------------
user >  삼겹살도 넉넉히 들어있고 김치랑 계란찜도 맛있었어요ㅎㅎ
sentiment: 긍정
실제답글 > 안녕하세요 :)
신자매 김치찜 &김치찌개를 찾아주셔서 진심으로 감사합니다🙏

삼겹살도 넉넉하고 김치, 계란찜 맛있다고 해주시니 넘 뿌듯합니당🧡 즐거운 주